In [1]:
import faiss
import numpy as np

index = faiss.read_index("docs.index")
embeddings = np.load("embeddings.npy")

In [4]:
import pickle

with open("docstore.pkl", "rb") as f:
    id_to_doc = pickle.load(f)

In [25]:
from sentence_transformers import SentenceTransformer

def retrieve_chunks(query, index, id_to_doc, k=20, score_threshold=0.6):
    model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
    query_emb = model.encode(
        query,
        normalize_embeddings=True
    ).reshape(1, -1)

    D, I = index.search(query_emb, k)

    docs = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1:
            continue
        if score < score_threshold:
            continue
        docs.append(id_to_doc[idx])

    return docs


In [74]:
query = "Give me the curl command to create a ticket."
chunks = retrieve_chunks(query, index, id_to_doc)

In [31]:
chunks

[Document(metadata={'anchor_id': 'view_a_ticket', 'parent_id': 'tickets', 'source': 'https://api.freshservice.com/', 'content_length': 5219, 'has_code': False, 'chunk_index': 154}, page_content="[Section: view_a_ticket]\n5. Get the associated offboarding information along with the ticket response.\n\ncurl -v -u api_key:X -X GET 'https://domain.freshservice.com/api/v2/tickets/20?include=offboarding_context'"),
 Document(metadata={'anchor_id': 'view_all_ticket', 'parent_id': 'tickets', 'source': 'https://api.freshservice.com/', 'content_length': 7163, 'has_code': False, 'chunk_index': 196}, page_content="[Section: view_all_ticket]\ncurl -v -u api_key:X -X GET 'https://domain.freshservice.com/api/v2/tickets?requester_id=1230&order_type=desc'\n\n3. Get the second page (tickets from 11-20) of a list of all tickets.\n\ncurl -v -u api_key:X -X GET 'https://domain.freshservice.com/api/v2/tickets?per_page=10&page=2'\n\n4. Get the first page of a list of tickets that have shown any activity sinc

In [75]:
from collections import defaultdict

def group_by_parent(docs):
    grouped = defaultdict(list)
    for doc in docs:
        grouped[doc.metadata["parent_id"]].append(doc)
    return grouped


In [76]:
grouped_docs = group_by_parent(chunks)

In [77]:
def build_context(grouped_docs, max_chars=5000):
    context_parts = []

    for parent, docs in grouped_docs.items():
        section_text = "\n".join(d.page_content for d in docs)
        block = f"### Section: {parent}\n{section_text}"
        context_parts.append(block)

    context = "\n\n".join(context_parts)
    return context[:max_chars]  # hard cap


In [78]:
context = build_context(grouped_docs)

In [79]:
context

'### Section: tickets\n[Section: create_ticket]\nSample code | Curl\ncurl -v -u api_key:X -F \'attachments[]=@/Users/user/Desktop/api_attach.png\' -F \'subject=Support Needed...\' -F \'description=Details about the issue...\' -F \'email=tom@outerspace.com\' -F \'priority=1\' -F \'status=2\' -F \'workspace_id=3\' -X POST \'https://domain.freshservice.com/api/v2/tickets\'\nResponse\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\n18\n19\n20\n21\n22\n23\n24\n25\n26\n27\n28\n29\n30\n31\n32\n33\n34\n35\n36\n37\n38\n39\n40\n41\n42\n43\n44\n45\n46\n47\n48\n49\n50\n51\n52\n\t{\n   "ticket": {\n[Section: create_ticket]\nSample code | Curl\ncurl -v -u api_key:X -H "Content-Type: application/json" -d \'{ "description": "Create ticket with assets linked", "status": 2, "email": "sample@freshservice.com", "priority": 1, "subject": "Create ticket with assets linked", "assets": [{ "display_id": 8 }, { "display_id": 9 }] "workspace_id": 3}\' -X POST \'https://domain.freshservice.com/api/v2/t

In [ ]:
SYSTEM_PROMPT = """
You are a technical assistant.
Answer ONLY using the provided context. Explain the answer in detail unsing the context.
If the answer is not present in the context, say:
"I don't have enough information in the provided documents."
Do not use prior knowledge.
"""

def build_prompt(context):
    return f"""
{SYSTEM_PROMPT}

Context:
{context}

Answer:
"""

In [86]:
prompt = build_prompt(context)

In [82]:
from dotenv import load_dotenv
import os

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise RuntimeError("GOOGLE_API_KEY not found in .env")

In [83]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
    max_tokens=2500,
    timeout=None,
    max_retries=2
)



In [87]:
messages = [
    (
        "system",
        prompt
    ),
    ("human", query),
]
ai_msg = model.invoke(messages)
print(ai_msg.content)

To create a ticket, you can use a `POST` request to the `/api/v2/tickets` endpoint.

Here's an example of a curl command that creates a ticket with a subject, description, sender email, priority, status, workspace ID, and an attachment:

```bash
curl -v -u api_key:X -F 'attachments[]=@/Users/user/Desktop/api_attach.png' -F 'subject=Support Needed...' -F 'description=Details about the issue...' -F 'email=tom@outerspace.com' -F 'priority=1' -F 'status=2' -F 'workspace_id=3' -X POST 'https://domain.freshservice.com/api/v2/tickets'
```

In this command:
*   `-v` provides verbose output.
*   `-u api_key:X` specifies the authentication using your API key.
*   `-F 'attachments[]=@/Users/user/Desktop/api_attach.png'` is used to attach a file.
*   `-F 'subject=Support Needed...'` sets the subject of the ticket.
*   `-F 'description=Details about the issue...'` provides the details of the problem.
*   `-F 'email=tom@outerspace.com'` specifies the email of the requester.
*   `-F 'priority=1'` set